# Neural Network Smartphone Activity Detector

In this activity, you will train a neural network to use smartphone data to predict the activity of the user. 

This dataset has already been separated into input features and target activities. Additional information on the dataset can be found here. 

http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

### Data Pre-Processing

Prepare the data for the neural network. This includes splitting the data into a training and testing dataset, Scaling the data, and encoding the categorical target values

In [16]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
# Read the input features into `X`
X = pd.read_csv(Path("../Resources/features.csv"), header=None)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,...,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,...,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092,0.180261,-0.047436
2,0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,...,0.440079,-0.404427,-0.761847,-0.118559,0.177899,0.100699,0.808529,-0.849230,0.180610,-0.042271
3,0.039785,-0.011809,-0.028916,-0.996194,-0.988569,-0.993256,-0.996994,-0.988526,-0.993135,-0.798477,...,0.430891,-0.138373,-0.491604,-0.036788,-0.012892,0.640011,-0.485366,-0.848947,0.181907,-0.040826
4,0.038758,-0.002289,-0.023863,-0.998241,-0.986774,-0.993115,-0.998216,-0.986479,-0.993825,-0.801982,...,0.137735,-0.366214,-0.702490,0.123320,0.122542,0.693578,-0.615971,-0.848164,0.185124,-0.037080


In [18]:
# Read the target values into `y`
y = pd.read_csv(Path("../Resources/target.csv"))
y.head()

,activity
0,standing
1,standing
2,standing
3,standing
4,standing


In [19]:
y.activity.value_counts()

standing              1423
laying                1413
sitting               1293
walking               1226
walking_upstairs      1073
walking_downstairs     987
stand_to_lie            90
sit_to_lie              75
lie_to_sit              60
lie_to_stand            57
stand_to_sit            47
sit_to_stand            23
Name: activity, dtype: int64

In [20]:
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [21]:
# Apply One-hot encoding to the target labels
enc = OneHotEncoder()
enc.fit(y_train)

encoded_y_train = enc.transform(y_train).toarray()
encoded_y_test = enc.transform(y_test).toarray()
encoded_y_train.shape

(5825, 12)

In [22]:
# Scale the training and testing input features using StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a Deep Neural Network

In [23]:
# Create a sequential model
nn = Sequential()
number_inputs = 561  #X_train_scaled.shape[1]
number_hidden_nodes = 100

In [24]:
# Add the first layer where the input dimensions are the 561 columns of the training data
nn.add(Dense(units=number_hidden_nodes, activation="relu", input_dim=number_inputs))

In [25]:
# Add output layer using 12 output nodes. 
# HINT: Use `softmax` as the activation 
number_classes = 12 #y_train.activity.value.counts()
nn.add(Dense(units=number_classes, activation="softmax"))


In [26]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Print the model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               56200     
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1212      
Total params: 57,412
Trainable params: 57,412
Non-trainable params: 0
_________________________________________________________________


In [28]:
X_train_scaled.shape
encoded_y_train.shape

(5825, 12)

In [29]:
# Use the training data to fit (train) the model
# @NOTE: Experiment with the number of training epochs to find the 
#minimum iterations required to achieve a good accuracy
model_activity = nn.fit(X_train_scaled, encoded_y_train, epochs=100, shuffle=True,
    verbose=2)

Epoch 1/100
183/183 - 1s - loss: 0.3844 - accuracy: 0.8675
Epoch 2/100
183/183 - 1s - loss: 0.1384 - accuracy: 0.9528
Epoch 3/100
183/183 - 1s - loss: 0.0906 - accuracy: 0.9667
Epoch 4/100
183/183 - 1s - loss: 0.0732 - accuracy: 0.9765
Epoch 5/100
183/183 - 1s - loss: 0.0530 - accuracy: 0.9823
Epoch 6/100
183/183 - 1s - loss: 0.0505 - accuracy: 0.9837
Epoch 7/100
183/183 - 1s - loss: 0.0429 - accuracy: 0.9845
Epoch 8/100
183/183 - 1s - loss: 0.0358 - accuracy: 0.9895
Epoch 9/100
183/183 - 1s - loss: 0.0372 - accuracy: 0.9866
Epoch 10/100
183/183 - 1s - loss: 0.0286 - accuracy: 0.9909
Epoch 11/100
183/183 - 1s - loss: 0.0234 - accuracy: 0.9923
Epoch 12/100
183/183 - 1s - loss: 0.0282 - accuracy: 0.9892
Epoch 13/100
183/183 - 1s - loss: 0.0207 - accuracy: 0.9928
Epoch 14/100
183/183 - 1s - loss: 0.0249 - accuracy: 0.9911
Epoch 15/100
183/183 - 1s - loss: 0.0156 - accuracy: 0.9948
Epoch 16/100
183/183 - 1s - loss: 0.0147 - accuracy: 0.9952
Epoch 17/100
183/183 - 1s - loss: 0.0153 - accura

# Evaluate the Model

In [32]:
# Evaluate the model using the testing data
model_activity_loss, model_activity_accuracy = nn.evaluate(
    X_test_scaled, encoded_y_test, verbose=2
)
print(f"Loss: {model_activity_loss}, Accuracy: {model_activity_accuracy}")

61/61 - 0s - loss: 0.1462 - accuracy: 0.9748
Loss: 0.14621859788894653, Accuracy: 0.9747682809829712


In [33]:
# Make predictions
# enc=OneHotEncoder()
# enc=enc.fit(y_test)
predicted = nn.predict(X_test_scaled)
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

,Actual,Predicted
0,walking_upstairs,walking_upstairs
1,walking,walking
2,walking_downstairs,walking_downstairs
3,laying,laying
4,walking_upstairs,walking_upstairs
5,sitting,sitting
6,standing,standing
7,walking_downstairs,walking_downstairs
8,walking_downstairs,walking_downstairs
9,walking_downstairs,walking_downstairs


In [34]:
# Print the Classification Report
from sklearn.metrics import classification_report
print(classification_report(results.Actual, results.Predicted))

                    precision    recall  f1-score   support

            laying       1.00      1.00      1.00       331
        lie_to_sit       0.65      0.71      0.68        21
      lie_to_stand       0.57      0.53      0.55        15
        sit_to_lie       0.65      0.72      0.68        18
      sit_to_stand       0.83      1.00      0.91         5
           sitting       0.98      0.97      0.97       355
      stand_to_lie       0.72      0.65      0.68        20
      stand_to_sit       1.00      0.89      0.94        18
          standing       0.97      0.98      0.97       361
           walking       1.00      1.00      1.00       305
walking_downstairs       1.00      1.00      1.00       244
  walking_upstairs       1.00      1.00      1.00       249

          accuracy                           0.97      1942
         macro avg       0.86      0.87      0.87      1942
      weighted avg       0.98      0.97      0.97      1942

